# User-player regression
Regressions targeting sentiment with covariates from both users and players. Single rows are determined by user-player-year triplet key

### Imports / load

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
import numpy as np
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [2]:
nba_df = pd.read_csv('c:/Users/map22/Google Drive/sentiment_nba/nba_user_player_sentiment.tsv', sep='\t')
nba_df = nba_df.dropna(subset=['Race', 'PPG']) # get some name matches for years players weren't playing / coaches
nba_df['rookie'] = nba_df['experience']<=1
nba_df = nba_df.replace({'M':'B', 'L':'W'}).query('Race == "W" or Race == "B"')

C:\Users\map22\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2785: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


#### Get salary residuals for NBA

In [5]:
nba_cov_df = pd.read_csv('modeling_data/nba_player_model_data.tsv', sep='\t')
nba_cov_df['rookie_contract'] = nba_cov_df['experience'] <=4

In [6]:
salary_model = smf.wls( formula = 'standard_salary ~ rookie + MP + PPG + ThreePP + DWS + TRBP * height_dummies + AST + BLKP * height_dummies + STLP + TOVP',
                data = nba_cov_df, weights = nba_cov_df['G']).fit()

In [7]:
nba_cov_df['pred_std_salary'] = salary_model.predict(nba_cov_df)
nba_cov_df['std_salary_resid'] = nba_cov_df['pred_std_salary']- nba_cov_df['standard_salary']
nba_cov_df.loc[ nba_cov_df['G'] <20, 'std_salary_resid'] = 0

In [8]:
nba_df = nba_df.merge(nba_cov_df[['Player','year', 'std_salary_resid']],
                      on=['Player', 'year'], how = 'left')

In [9]:
for col in ['FTr','TOVP', 'standard_salary', 'std_salary_resid', 'clinton_vote_lead', 'white_black_diff', 'total_population' ]:
    nba_df[col] = nba_df[col].fillna(nba_df[col].mean())

In [10]:
nba_df['demean_PPG'] = nba_df['PPG'] - nba_df['PPG'].mean()
nba_df['demean_clinton'] = nba_df['clinton_vote_lead'] - nba_df['clinton_vote_lead'].mean()
nba_df['demean_race_diff'] = nba_df['white_black_diff'] - nba_df['white_black_diff'].mean()

#### NFL

In [9]:
nfl_df = pd.read_csv('c:/Users/map22/Google Drive/sentiment_nba/nfl_user_player_sentiment.tsv', sep='\t')
nfl_df = nfl_df.dropna(subset=['race']) # get some name matches for years players weren't playing / coaches
nfl_df['rookie'] = nfl_df['experience'] <=1
nfl_df['race'] = nfl_df['race'].replace({'L':'B', 'S':'B', 'M':'B'})

In [10]:
nfl_cov_df = pd.read_csv('modeling_data/nfl_model_data.tsv', sep='\t')
nfl_cov_df['rookie_contract'] = nfl_cov_df['experience'] <=4

In [11]:
salary_model = smf.wls( formula = 'standard_salary ~ rookie_contract + position + z_DVOA',
                data = nfl_cov_df, weights = 1).fit()

In [12]:
nfl_cov_df['pred_std_salary'] = salary_model.predict(nfl_cov_df)
nfl_cov_df['std_salary_resid'] = nfl_cov_df['pred_std_salary']- nfl_cov_df['standard_salary']

In [13]:
nfl_df = nfl_df.merge(nfl_cov_df[['Player','year', 'std_salary_resid']],
                      on=['Player', 'year'], how = 'left')

In [14]:
for col in [ 'std_salary_resid', 'clinton_vote_lead', 'white_black_diff','total_population' ]:
    nfl_df[col] = nfl_df[col].fillna(nfl_df[col].mean())

In [15]:
nfl_df['demean_DVOA'] = nfl_df['z_DVOA'] - nfl_df['z_DVOA'].mean()
nfl_df['demean_clinton'] = nfl_df['clinton_vote_lead'] - nfl_df['clinton_vote_lead'].mean()
nfl_df['demean_race_diff'] = nfl_df['white_black_diff'] - nfl_df['white_black_diff'].mean()

## NBA
#### Points + MP only model - unclustered

In [11]:
model = smf.wls( formula = 'compound_mean ~ MP + PPG', \
#                       ' PPG +  + total_population+  * white_black_diff + C(Race) * clinton_vote_lead',
                data = nba_df,
               weights = 1,# / (nba_df['compound_mean_std'] / np.sqrt(fit_df['user_count'])),
#                missing='raise'   
               ).fit()

In [12]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.000
Model:                            WLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     104.6
Date:                Sun, 10 Feb 2019   Prob (F-statistic):           3.78e-46
Time:                        13:44:57   Log-Likelihood:            -2.6943e+05
No. Observations:              848905   AIC:                         5.389e+05
Df Residuals:                  848902   BIC:                         5.389e+05
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0699      0.001     73.825      0.000       0.068       0.072
MP         -1.036e-06   6.52e-07     -1.589      0.112   -2.31e-06    2.42e-07
PPG            0.0007   6.46e-05     11.085      0.000       0.001       0.001
==============================================================================
Omnibus:                     1034.625   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1158.714
Skew:                          -0.044   Prob(JB):                    2.45e-252
Kurtosis:                       3.158   Cond. No.                     5.48e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.48e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Points + MP only model - clustered

In [39]:
model = smf.wls( formula = 'compound_mean ~ MP + PPG', \
#                       ' PPG +  + total_population+  * white_black_diff + C(Race) * clinton_vote_lead',
                data = nba_df,
               weights = 1,# / (nba_df['compound_mean_std'] / np.sqrt(fit_df['user_count'])),
#                missing='raise'   
               ).fit(cov_type='cluster', cov_kwds={'groups' : nba_df['Player']})

In [40]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.000
Model:                            WLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     4.224
Date:                Sun, 10 Feb 2019   Prob (F-statistic):             0.0150
Time:                        13:54:25   Log-Likelihood:            -2.6943e+05
No. Observations:              848905   AIC:                         5.389e+05
Df Residuals:                  848902   BIC:                         5.389e+05
Df Model:                           2                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0699      0.003     21.002      0.000       0.063       0.076
MP         -1.036e-06   2.93e-06     -0.354      0.723   -6.77e-06     4.7e-06
PPG            0.0007      0.000      1.985      0.047    8.87e-06       0.001
==============================================================================
Omnibus:                     1034.625   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1158.714
Skew:                          -0.044   Prob(JB):                    2.45e-252
Kurtosis:                       3.158   Cond. No.                     5.48e+03
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
[2] The condition number is large, 5.48e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Full performance

In [21]:
model = smf.wls( formula = 'compound_mean ~ MP + PPG + ThreePP + DWS + TRBP + FTr + ASTP + BLKP + STLP + TOVP', \
#                       ' PPG +  + total_population+  * white_black_diff + C(Race) * clinton_vote_lead',
                data = nba_df,
               weights = 1,# / (nba_df['compound_mean_std'] / np.sqrt(fit_df['user_count'])),
#                missing='raise'   
               ).fit(cov_type='cluster', cov_kwds={'groups' : nba_df['Player']})

In [22]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.001
Model:                            WLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     2.866
Date:                Sun, 10 Feb 2019   Prob (F-statistic):            0.00161
Time:                        13:46:54   Log-Likelihood:            -2.6931e+05
No. Observations:              848905   AIC:                         5.386e+05
Df Residuals:                  848894   BIC:                         5.388e+05
Df Model:                          10                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0732      0.009      8.554      0.000       0.056       0.090
MP          1.812e-07   3.26e-06      0.056      0.956   -6.21e-06    6.57e-06
PPG            0.0009      0.000      2.314      0.021       0.000       0.002
ThreePP        0.0059      0.015      0.385      0.701      -0.024       0.036
DWS           -0.0013      0.002     -0.516      0.606      -0.006       0.004
TRBP          -0.0004      0.000     -0.704      0.482      -0.001       0.001
FTr           -0.0169      0.009     -1.787      0.074      -0.035       0.002
ASTP          -0.0002      0.000     -1.412      0.158      -0.001    8.46e-05
BLKP           0.0021      0.001      1.502      0.133      -0.001       0.005
STLP           0.0044      0.002      2.111      0.035       0.000       0.008
TOVP          -0.0004      0.000     -1.249      0.212      -0.001       0.000
==============================================================================
Omnibus:                     1027.798   Durbin-Watson:                   1.981
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1150.734
Skew:                          -0.044   Prob(JB):                    1.32e-250
Kurtosis:                       3.158   Cond. No.                     2.15e+04
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
[2] The condition number is large, 2.15e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Full performance + height / age

In [13]:
model = smf.wls( formula = 'compound_mean ~ MP + PPG + ThreePP + DWS + TRBP + FTr + ASTP + BLKP + STLP + TOVP' \
                       ' + C(height_dummies) + rookie + youth + oldness',
                data = nba_df,
               weights = 1,# / (nba_df['compound_mean_std'] / np.sqrt(fit_df['user_count'])),
#                missing='raise'   
               ).fit(cov_type='cluster', cov_kwds={'groups' : nba_df['Player']})

In [14]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.002
Model:                            WLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     8.965
Date:                Sun, 10 Feb 2019   Prob (F-statistic):           1.48e-19
Time:                        13:46:01   Log-Likelihood:            -2.6890e+05
No. Observations:              848905   AIC:                         5.378e+05
Df Residuals:                  848889   BIC:                         5.380e+05
Df Model:                          15                                         
Covariance Type:              cluster                                         
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.0592      0.007      8.034      0.000       0.045       0.074
C(height_dummies)[T.short]     0.0069      0.005      1.404      0.160      -0.003       0.016
C(height_dummies)[T.tall]     -0.0001      0.004     -0.032      0.974      -0.008       0.008
rookie[T.True]                 0.0199      0.004      5.409      0.000       0.013       0.027
MP                         -2.629e-06      3e-06     -0.876      0.381   -8.51e-06    3.25e-06
PPG                            0.0014      0.000      3.492      0.000       0.001       0.002
ThreePP                        0.0102      0.013      0.791      0.429      -0.015       0.035
DWS                            0.0008      0.002      0.383      0.702      -0.003       0.005
TRBP                          -0.0004      0.000     -0.821      0.412      -0.001       0.000
FTr                           -0.0138      0.009     -1.546      0.122      -0.031       0.004
ASTP                          -0.0003      0.000     -1.434      0.152      -0.001    9.37e-05
BLKP                           0.0009      0.001      0.752      0.452      -0.001       0.003
STLP                           0.0029      0.002      1.448      0.147      -0.001       0.007
TOVP                          -0.0004      0.000     -1.109      0.268      -0.001       0.000
youth                          0.0026      0.001      3.504      0.000       0.001       0.004
oldness                        0.0014      0.001      2.039      0.041    5.61e-05       0.003
==============================================================================
Omnibus:                     1019.358   Durbin-Watson:                   1.982
Prob(Omnibus):                  0.000   Jarque-Bera (JB):             1143.163
Skew:                          -0.043   Prob(JB):                    5.83e-249
Kurtosis:                       3.158   Cond. No.                     2.19e+04
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
[2] The condition number is large, 2.19e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Full performance + height / age + salary

In [25]:
model = smf.wls( formula = 'compound_mean ~ MP + PPG + ThreePP + DWS + TRBP + FTr + ASTP + BLKP + STLP + TOVP' \
                       ' + C(height_dummies) + rookie + youth + oldness + standard_salary + std_salary_resid',
                data = nba_df,
               weights = np.sqrt(nba_df['comment_count']),# / (nba_df['compound_mean_std'] / np.sqrt(fit_df['user_count'])),
#                missing='raise'   
               ).fit(cov_type='cluster', cov_kwds={'groups' : nba_df['Player']})

In [26]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.002
Model:                            WLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     7.786
Date:                Sun, 10 Feb 2019   Prob (F-statistic):           2.64e-18
Time:                        13:48:57   Log-Likelihood:            -2.4534e+05
No. Observations:              848905   AIC:                         4.907e+05
Df Residuals:                  848887   BIC:                         4.909e+05
Df Model:                          17                                         
Covariance Type:              cluster                                         
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.0543      0.014      3.844      0.000       0.027       0.082
C(height_dummies)[T.short]     0.0075      0.005      1.525      0.127      -0.002       0.017
C(height_dummies)[T.tall]     -0.0007      0.004     -0.169      0.866      -0.009       0.008
rookie[T.True]                 0.0207      0.004      4.736      0.000       0.012       0.029
MP                         -3.616e-06   3.11e-06     -1.161      0.246   -9.72e-06    2.49e-06
PPG                            0.0013      0.001      2.552      0.011       0.000       0.002
ThreePP                        0.0109      0.013      0.810      0.418      -0.015       0.037
DWS                            0.0013      0.002      0.621      0.535      -0.003       0.005
TRBP                          -0.0004      0.000     -0.826      0.409      -0.001       0.001
FTr                           -0.0131      0.010     -1.346      0.178      -0.032       0.006
ASTP                          -0.0003      0.000     -1.444      0.149      -0.001    9.48e-05
BLKP                           0.0007      0.001      0.588      0.557      -0.002       0.003
STLP                           0.0029      0.002      1.424      0.154      -0.001       0.007
TOVP                          -0.0004      0.000     -1.112      0.266      -0.001       0.000
youth                          0.0028      0.001      3.547      0.000       0.001       0.004
oldness                        0.0014      0.001      1.881      0.060   -5.94e-05       0.003
standard_salary                0.0001      0.000      0.429      0.668      -0.000       0.001
std_salary_resid            5.514e-05      0.000      0.177      0.859      -0.001       0.001
==============================================================================
Omnibus:                      265.066   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              266.889
Skew:                          -0.039   Prob(JB):                     1.11e-58
Kurtosis:                       3.037   Cond. No.                     2.91e+04
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
[2] The condition number is large, 2.91e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Full performance + height / age + salary + Team wins

In [27]:
model = smf.wls( formula = 'compound_mean ~ MP + PPG + ThreePP + DWS + TRBP + FTr + ASTP + BLKP + STLP + TOVP' \
                       ' + C(height_dummies) + rookie + youth + oldness + standard_salary + std_salary_resid + Wins',
                data = nba_df,
               weights = np.sqrt(nba_df['comment_count']),# / (nba_df['compound_mean_std'] / np.sqrt(fit_df['user_count'])),
#                missing='raise'   
               ).fit(cov_type='cluster', cov_kwds={'groups' : nba_df['Player']})

In [28]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.002
Model:                            WLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     7.456
Date:                Sun, 10 Feb 2019   Prob (F-statistic):           3.52e-18
Time:                        13:50:05   Log-Likelihood:            -2.4534e+05
No. Observations:              848905   AIC:                         4.907e+05
Df Residuals:                  848886   BIC:                         4.909e+05
Df Model:                          18                                         
Covariance Type:              cluster                                         
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.0576      0.015      3.731      0.000       0.027       0.088
C(height_dummies)[T.short]     0.0075      0.005      1.525      0.127      -0.002       0.017
C(height_dummies)[T.tall]     -0.0008      0.004     -0.185      0.853      -0.009       0.008
rookie[T.True]                 0.0208      0.004      4.743      0.000       0.012       0.029
MP                         -4.208e-06   3.22e-06     -1.307      0.191   -1.05e-05     2.1e-06
PPG                            0.0013      0.001      2.495      0.013       0.000       0.002
ThreePP                        0.0116      0.014      0.837      0.403      -0.016       0.039
DWS                            0.0020      0.002      0.859      0.390      -0.003       0.006
TRBP                          -0.0004      0.000     -0.905      0.365      -0.001       0.001
FTr                           -0.0132      0.010     -1.363      0.173      -0.032       0.006
ASTP                          -0.0003      0.000     -1.447      0.148      -0.001    9.48e-05
BLKP                           0.0007      0.001      0.569      0.569      -0.002       0.003
STLP                           0.0028      0.002      1.317      0.188      -0.001       0.007
TOVP                          -0.0004      0.000     -1.113      0.266      -0.001       0.000
youth                          0.0028      0.001      3.520      0.000       0.001       0.004
oldness                        0.0014      0.001      1.828      0.068   -9.97e-05       0.003
standard_salary                0.0001      0.000      0.481      0.630      -0.000       0.001
std_salary_resid            7.366e-05      0.000      0.239      0.811      -0.001       0.001
Wins                       -6.795e-05      0.000     -0.473      0.636      -0.000       0.000
==============================================================================
Omnibus:                      264.924   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              266.737
Skew:                          -0.039   Prob(JB):                     1.20e-58
Kurtosis:                       3.037   Cond. No.                     3.03e+04
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
[2] The condition number is large, 3.03e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Full performance + height / age + salary + race of player

In [29]:
model = smf.wls( formula = 'compound_mean ~ MP + PPG + ThreePP + DWS + TRBP + FTr + ASTP + BLKP + STLP + TOVP' \
                       ' + C(height_dummies) + rookie + youth + oldness + standard_salary + std_salary_resid + C(Race)',
                data = nba_df,
               weights = np.sqrt(nba_df['comment_count']),# / (nba_df['compound_mean_std'] / np.sqrt(fit_df['user_count'])),
#                missing='raise'   
               ).fit(cov_type='cluster', cov_kwds={'groups' : nba_df['Player']})

In [30]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.002
Model:                            WLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     7.459
Date:                Sun, 10 Feb 2019   Prob (F-statistic):           3.46e-18
Time:                        13:51:04   Log-Likelihood:            -2.4534e+05
No. Observations:              848905   AIC:                         4.907e+05
Df Residuals:                  848886   BIC:                         4.909e+05
Df Model:                          18                                         
Covariance Type:              cluster                                         
==============================================================================================
                                 coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------------------
Intercept                      0.0542      0.014      3.858      0.000       0.027       0.082
C(height_dummies)[T.short]     0.0076      0.005      1.532      0.126      -0.002       0.017
C(height_dummies)[T.tall]     -0.0008      0.004     -0.184      0.854      -0.009       0.007
rookie[T.True]                 0.0207      0.004      4.708      0.000       0.012       0.029
C(Race)[T.W]                   0.0004      0.005      0.080      0.936      -0.009       0.010
MP                          -3.61e-06   3.11e-06     -1.160      0.246   -9.71e-06    2.49e-06
PPG                            0.0014      0.001      2.523      0.012       0.000       0.002
ThreePP                        0.0108      0.014      0.771      0.441      -0.017       0.038
DWS                            0.0013      0.002      0.617      0.537      -0.003       0.006
TRBP                          -0.0004      0.000     -0.844      0.399      -0.001       0.001
FTr                           -0.0131      0.010     -1.352      0.176      -0.032       0.006
ASTP                          -0.0003      0.000     -1.444      0.149      -0.001    9.49e-05
BLKP                           0.0007      0.001      0.628      0.530      -0.002       0.003
STLP                           0.0029      0.002      1.369      0.171      -0.001       0.007
TOVP                          -0.0004      0.000     -1.112      0.266      -0.001       0.000
youth                          0.0028      0.001      3.557      0.000       0.001       0.004
oldness                        0.0014      0.001      1.892      0.058   -5.05e-05       0.003
standard_salary                0.0001      0.000      0.426      0.670      -0.000       0.001
std_salary_resid            5.257e-05      0.000      0.169      0.866      -0.001       0.001
==============================================================================
Omnibus:                      265.119   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              266.941
Skew:                          -0.039   Prob(JB):                     1.08e-58
Kurtosis:                       3.037   Cond. No.                     2.91e+04
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
[2] The condition number is large, 2.91e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

#### Full performance + height / age + salary + race of player + city vote

In [43]:
model = smf.wls( formula = 'compound_mean ~ MP + ThreePP + DWS + TRBP + FTr + ASTP + BLKP + STLP + TOVP' \
                       ' + C(height_dummies) + rookie + youth + oldness + standard_salary + std_salary_resid + Wins + ' \
                'team_match + C(Race) * demean_clinton +total_population + C(Race) * demean_PPG',
                data = nba_df,
               weights = np.sqrt(nba_df['comment_count']),# / (nba_df['compound_mean_std'] / np.sqrt(fit_df['user_count'])),
#                missing='raise'   
               ).fit(cov_type='cluster', cov_kwds={'groups' : nba_df['Player']})

In [44]:
model.summary()

C:\Users\map22\Anaconda3\lib\site-packages\statsmodels\base\model.py:1532: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 24, but rank is 23
  'rank is %d' % (J, J_), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.002
Model:                            WLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     8.448
Date:                Sun, 10 Feb 2019   Prob (F-statistic):           9.92e-26
Time:                        14:01:24   Log-Likelihood:            -2.4529e+05
No. Observations:              848905   AIC:                         4.906e+05
Df Residuals:                  848880   BIC:                         4.909e+05
Df Model:                          24                                         
Covariance Type:              cluster                                         
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                       0.0774      0.021      3.637      0.000       0.036       0.119
C(height_dummies)[T.short]      0.0071      0.005      1.419      0.156      -0.003       0.017
C(height_dummies)[T.tall]      -0.0003      0.004     -0.062      0.951      -0.008       0.008
rookie[T.True]                  0.0217      0.005      4.779      0.000       0.013       0.031
team_match[T.True]              0.0024      0.002      1.493      0.136      -0.001       0.006
C(Race)[T.W]                    0.0081      0.005      1.640      0.101      -0.002       0.018
MP                          -5.007e-06   3.31e-06     -1.511      0.131   -1.15e-05    1.49e-06
ThreePP                         0.0111      0.014      0.801      0.423      -0.016       0.038
DWS                             0.0019      0.002      0.830      0.406      -0.003       0.006
TRBP                           -0.0005      0.000     -0.977      0.328      -0.001       0.000
FTr                            -0.0113      0.010     -1.159      0.246      -0.030       0.008
ASTP                           -0.0002      0.000     -1.298      0.194      -0.001       0.000
BLKP                            0.0005      0.001      0.421      0.673      -0.002       0.003
STLP                            0.0028      0.002      1.326      0.185      -0.001       0.007
TOVP                           -0.0003      0.000     -0.970      0.332      -0.001       0.000
youth                           0.0026      0.001      3.181      0.001       0.001       0.004
oldness                         0.0012      0.001      1.705      0.088      -0.000       0.003
standard_salary                 0.0002      0.000      0.531      0.595      -0.000       0.001
std_salary_resid             7.703e-05      0.000      0.246      0.806      -0.001       0.001
Wins                        -4.443e-05      0.000     -0.323      0.747      -0.000       0.000
demean_clinton                  0.0096      0.003      3.160      0.002       0.004       0.015
C(Race)[T.W]:demean_clinton    -0.0048      0.007     -0.668      0.504      -0.019       0.009
total_population            -1.906e-10   1.13e-10     -1.693      0.091   -4.11e-10    3.01e-11
demean_PPG                      0.0012      0.001      2.183      0.029       0.000       0.002
C(Race)[T.W]:demean_PPG         0.0016      0.001      2.605      0.009       0.000       0.003
==============================================================================
Omnibus:                      260.874   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              262.629
Skew:                          -0.039   Prob(JB):                     9.35e-58
Kurtosis:                       3.036   Cond. No.                     1.31e+08


#### Full performance + height / age + salary + race of player + city demographics

In [35]:
model = smf.wls( formula = 'compound_mean ~ MP + ThreePP + DWS + TRBP + FTr + ASTP + BLKP + STLP + TOVP' \
                       ' + C(height_dummies) + rookie + youth + oldness + standard_salary + std_salary_resid + Wins + ' \
                'team_match + C(Race) * demean_race_diff +total_population + C(Race) * demean_PPG',
                data = nba_df,
               weights = np.sqrt(nba_df['comment_count']),# / (nba_df['compound_mean_std'] / np.sqrt(fit_df['user_count'])),
#                missing='raise'   
               ).fit(cov_type='cluster', cov_kwds={'groups' : nba_df['Player']})

In [36]:
model.summary()

C:\Users\map22\Anaconda3\lib\site-packages\statsmodels\base\model.py:1532: ValueWarning: covariance of constraints does not have full rank. The number of constraints is 24, but rank is 23
  'rank is %d' % (J, J_), ValueWarning)


<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.002
Model:                            WLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     8.000
Date:                Sun, 10 Feb 2019   Prob (F-statistic):           4.19e-24
Time:                        13:52:12   Log-Likelihood:            -2.4530e+05
No. Observations:              848905   AIC:                         4.906e+05
Df Residuals:                  848880   BIC:                         4.909e+05
Df Model:                          24                                         
Covariance Type:              cluster                                         
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         0.0760      0.021      3.589      0.000       0.034       0.117
C(height_dummies)[T.short]        0.0070      0.005      1.399      0.162      -0.003       0.017
C(height_dummies)[T.tall]        -0.0003      0.004     -0.063      0.950      -0.008       0.008
rookie[T.True]                    0.0217      0.005      4.784      0.000       0.013       0.031
team_match[T.True]                0.0024      0.002      1.513      0.130      -0.001       0.006
C(Race)[T.W]                      0.0080      0.005      1.619      0.105      -0.002       0.018
MP                            -4.908e-06   3.32e-06     -1.479      0.139   -1.14e-05     1.6e-06
ThreePP                           0.0111      0.014      0.805      0.421      -0.016       0.038
DWS                               0.0018      0.002      0.799      0.425      -0.003       0.006
TRBP                             -0.0004      0.000     -0.965      0.335      -0.001       0.000
FTr                              -0.0118      0.010     -1.199      0.230      -0.031       0.007
ASTP                             -0.0002      0.000     -1.288      0.198      -0.001       0.000
BLKP                              0.0005      0.001      0.452      0.651      -0.002       0.003
STLP                              0.0028      0.002      1.320      0.187      -0.001       0.007
TOVP                             -0.0003      0.000     -0.953      0.341      -0.001       0.000
youth                             0.0025      0.001      3.171      0.002       0.001       0.004
oldness                           0.0012      0.001      1.691      0.091      -0.000       0.003
standard_salary                   0.0002      0.000      0.523      0.601      -0.000       0.001
std_salary_resid               7.478e-05      0.000      0.239      0.811      -0.001       0.001
Wins                          -3.932e-05      0.000     -0.286      0.775      -0.000       0.000
demean_race_diff               1.672e-05   3.79e-05      0.442      0.659   -5.75e-05     9.1e-05
C(Race)[T.W]:demean_race_diff  7.832e-05   8.93e-05      0.877      0.381   -9.68e-05       0.000
total_population               2.731e-11   1.12e-10      0.244      0.808   -1.92e-10    2.47e-10
demean_PPG                        0.0012      0.001      2.190      0.029       0.000       0.002
C(Race)[T.W]:demean_PPG           0.0016      0.001      2.599      0.009       0.000       0.003
==============================================================================
Omnibus:                      260.009   Durbin-Watson:                   1.980
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              261.758
Skew:                          -0.039   Prob(JB):                     1.45e-57
Kurtosis:              

#### Race and PPG

In [41]:
model = smf.wls( formula = 'compound_mean ~ MP + C(Race) * demean_PPG',
                data = nba_df,
               weights = np.sqrt(nba_df['comment_count']),# / (nba_df['compound_mean_std'] / np.sqrt(fit_df['user_count'])),
#                missing='raise'   
               ).fit(cov_type='cluster', cov_kwds={'groups' : nba_df['Player']})

In [42]:
model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.001
Model:                            WLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     5.519
Date:                Sun, 10 Feb 2019   Prob (F-statistic):           0.000218
Time:                        14:00:32   Log-Likelihood:            -2.4586e+05
No. Observations:              848905   AIC:                         4.917e+05
Df Residuals:                  848900   BIC:                         4.918e+05
Df Model:                           4                                         
Covariance Type:              cluster                                         
===========================================================================================
                              coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------
Intercept                   0.0853      0.006     14.902      0.000       0.074       0.096
C(Race)[T.W]                0.0095      0.005      1.764      0.078      -0.001       0.020
MP                      -2.518e-06      3e-06     -0.838      0.402   -8.41e-06    3.37e-06
demean_PPG                  0.0007      0.000      1.915      0.056   -1.66e-05       0.001
C(Race)[T.W]:demean_PPG     0.0022      0.001      3.171      0.002       0.001       0.004
==============================================================================
Omnibus:                      282.469   Durbin-Watson:                   1.978
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              284.246
Skew:                          -0.041   Prob(JB):                     1.89e-62
Kurtosis:                       3.036   Cond. No.                     8.25e+03
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
[2] The condition number is large, 8.25e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## NFL

### z_DVOA only

In [23]:
nfl_model = smf.wls( formula = 'compound_mean ~ z_DVOA',#\
                       #' + white_black_diff * C(race) + clinton_vote_lead',',#'
                data = nfl_df, weights = 1 
               ).fit(cov_type='cluster', cov_kwds={'groups' : nfl_df['Player']})

In [24]:
nfl_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.000
Model:                            WLS   Adj. R-squared:                  0.000
Method:                 Least Squares   F-statistic:                     9.575
Date:                Mon, 24 Dec 2018   Prob (F-statistic):            0.00210
Time:                        08:34:59   Log-Likelihood:                -96293.
No. Observations:              292531   AIC:                         1.926e+05
Df Residuals:                  292529   BIC:                         1.926e+05
Df Model:                           1                                         
Covariance Type:              cluster                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0670      0.002     31.603      0.000       0.063       0.071
z_DVOA         0.0057      0.002      3.094      0.002       0.002       0.009
==============================================================================
Omnibus:                      237.333   Durbin-Watson:                   1.957
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              263.144
Skew:                          -0.031   Prob(JB):                     7.23e-58
Kurtosis:                       3.133   Cond. No.                         1.46
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
"""

### Performance + demo + position + salary

In [32]:
nfl_model = smf.wls( formula = 'compound_mean ~ z_DVOA + youth + oldness + rookie + C(position) + C(height_dummies) + Wins + standard_salary + std_salary_resid',#\
                       #' + white_black_diff * C(race) + clinton_vote_lead',',#'
                data = nfl_df, weights = 1, 
               ).fit(cov_type='cluster', cov_kwds={'groups' : nfl_df['Player']})

In [33]:
nfl_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.001
Model:                            WLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     5.489
Date:                Mon, 24 Dec 2018   Prob (F-statistic):           3.16e-08
Time:                        08:39:40   Log-Likelihood:                -96157.
No. Observations:              292531   AIC:                         1.923e+05
Df Residuals:                  292519   BIC:                         1.925e+05
Df Model:                          11                                         
Covariance Type:              cluster                                         
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                     0.0683      0.067      1.014      0.310      -0.064       0.200
rookie[T.True]                0.0172      0.007      2.316      0.021       0.003       0.032
C(position)[T.rb]             0.0077      0.006      1.273      0.203      -0.004       0.020
C(position)[T.te]            -0.0060      0.005     -1.188      0.235      -0.016       0.004
C(position)[T.wr]            -0.0063      0.006     -0.976      0.329      -0.019       0.006
C(height_dummies)[T.tall]     0.0041      0.006      0.727      0.468      -0.007       0.015
z_DVOA                        0.0074      0.002      3.963      0.000       0.004       0.011
youth                         0.0009      0.002      0.542      0.588      -0.002       0.004
oldness                      -0.0006      0.001     -0.764      0.445      -0.002       0.001
Wins                         -0.0001      0.001     -0.205      0.838      -0.001       0.001
standard_salary              -0.0004      0.004     -0.104      0.917      -0.009       0.008
std_salary_resid           7.146e-05      0.003      0.021      0.983      -0.006       0.007
==============================================================================
Omnibus:                      239.797   Durbin-Watson:                   1.958
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              265.519
Skew:                          -0.032   Prob(JB):                     2.20e-58
Kurtosis:                       3.133   Cond. No.                         879.
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
"""

### Performance + demo + position + salary + race

In [38]:
nfl_model = smf.wls( formula = 'compound_mean ~ z_DVOA + youth + oldness + rookie + C(position) + C(height_dummies) + Wins + standard_salary + std_salary_resid'\
                       '+ C(race)',#' + white_black_diff *  + clinton_vote_lead',',#'
                data = nfl_df, weights = 1, 
               ).fit(cov_type='cluster', cov_kwds={'groups' : nfl_df['Player']})

In [39]:
nfl_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.001
Model:                            WLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     5.241
Date:                Mon, 24 Dec 2018   Prob (F-statistic):           2.88e-08
Time:                        08:42:56   Log-Likelihood:                -96152.
No. Observations:              292531   AIC:                         1.923e+05
Df Residuals:                  292518   BIC:                         1.925e+05
Df Model:                          12                                         
Covariance Type:              cluster                                         
=============================================================================================
                                coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------------------
Intercept                     0.0667      0.067      0.995      0.320      -0.065       0.198
rookie[T.True]                0.0172      0.007      2.310      0.021       0.003       0.032
C(position)[T.rb]             0.0117      0.006      1.818      0.069      -0.001       0.024
C(position)[T.te]            -0.0058      0.005     -1.120      0.263      -0.016       0.004
C(position)[T.wr]            -0.0028      0.007     -0.403      0.687      -0.016       0.011
C(height_dummies)[T.tall]     0.0037      0.006      0.649      0.517      -0.007       0.015
C(race)[T.W]                  0.0058      0.006      1.025      0.306      -0.005       0.017
z_DVOA                        0.0072      0.002      3.891      0.000       0.004       0.011
youth                         0.0011      0.002      0.659      0.510      -0.002       0.004
oldness                      -0.0006      0.001     -0.818      0.413      -0.002       0.001
Wins                         -0.0001      0.001     -0.248      0.805      -0.001       0.001
standard_salary              -0.0006      0.004     -0.148      0.883      -0.009       0.008
std_salary_resid           4.982e-05      0.003      0.015      0.988      -0.006       0.007
==============================================================================
Omnibus:                      239.514   Durbin-Watson:                   1.958
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              265.232
Skew:                          -0.032   Prob(JB):                     2.54e-58
Kurtosis:                       3.133   Cond. No.                         880.
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
"""

### Performance + demo + position + salary + race + user + city vote

In [23]:
nfl_model = smf.wls( formula = 'compound_mean ~ C(race) * demean_DVOA + youth + oldness + rookie + C(position) + C(height_dummies) + Wins + standard_salary + std_salary_resid'\
                       '+ team_match + C(race) * demean_clinton + total_population',
                data = nfl_df, weights = 1, 
               ).fit(cov_type='cluster', cov_kwds={'groups' : nfl_df['Player']})

In [24]:
nfl_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.001
Model:                            WLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     7.067
Date:                Mon, 21 Jan 2019   Prob (F-statistic):           3.01e-15
Time:                        18:36:25   Log-Likelihood:                -96140.
No. Observations:              292531   AIC:                         1.923e+05
Df Residuals:                  292513   BIC:                         1.925e+05
Df Model:                          17                                         
Covariance Type:              cluster                                         
===============================================================================================
                                  coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                       0.0715      0.066      1.086      0.278      -0.058       0.200
C(race)[T.W]                    0.0067      0.006      1.198      0.231      -0.004       0.018
rookie[T.True]                  0.0176      0.007      2.462      0.014       0.004       0.032
C(position)[T.rb]               0.0120      0.006      1.865      0.062      -0.001       0.025
C(position)[T.te]              -0.0070      0.005     -1.321      0.186      -0.017       0.003
C(position)[T.wr]              -0.0024      0.007     -0.352      0.725      -0.015       0.011
C(height_dummies)[T.tall]       0.0040      0.006      0.711      0.477      -0.007       0.015
team_match[T.True]              0.0023      0.002      0.912      0.362      -0.003       0.007
demean_DVOA                     0.0051      0.004      1.433      0.152      -0.002       0.012
C(race)[T.W]:demean_DVOA        0.0039      0.004      0.997      0.319      -0.004       0.012
youth                           0.0011      0.002      0.692      0.489      -0.002       0.004
oldness                        -0.0007      0.001     -0.897      0.370      -0.002       0.001
Wins                           -0.0002      0.001     -0.401      0.689      -0.001       0.001
standard_salary                -0.0007      0.004     -0.164      0.870      -0.009       0.007
std_salary_resid               -0.0002      0.003     -0.061      0.951      -0.007       0.006
demean_clinton                  0.0051      0.005      0.952      0.341      -0.005       0.016
C(race)[T.W]:demean_clinton     0.0073      0.006      1.153      0.249      -0.005       0.020
total_population            -4.222e-10   1.76e-10     -2.401      0.016   -7.67e-10   -7.76e-11
==============================================================================
Omnibus:                      237.391   Durbin-Watson:                   1.958
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              263.085
Skew:                          -0.031   Prob(JB):                     7.45e-58
Kurtosis:                       3.133   Cond. No.                     3.26e+08
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
[2] The condition number is large, 3.26e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

### Performance + demo + position + salary + race + user + city demo

In [29]:
nfl_model = smf.wls( formula = 'compound_mean ~ C(race) * demean_DVOA + youth + oldness + rookie + C(position) + C(height_dummies) + Wins + standard_salary + std_salary_resid'\
                       '+ team_match + C(race) * demean_race_diff + total_population',
                data = nfl_df, weights = 1, 
               ).fit(cov_type='cluster', cov_kwds={'groups' : nfl_df['Player']})

In [30]:
nfl_model.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            WLS Regression Results                            
==============================================================================
Dep. Variable:          compound_mean   R-squared:                       0.001
Model:                            WLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     5.194
Date:                Mon, 21 Jan 2019   Prob (F-statistic):           1.86e-10
Time:                        18:38:22   Log-Likelihood:                -96144.
No. Observations:              292531   AIC:                         1.923e+05
Df Residuals:                  292513   BIC:                         1.925e+05
Df Model:                          17                                         
Covariance Type:              cluster                                         
=================================================================================================
                                    coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------------------
Intercept                         0.0698      0.066      1.054      0.292      -0.060       0.200
C(race)[T.W]                      0.0067      0.006      1.181      0.238      -0.004       0.018
rookie[T.True]                    0.0175      0.007      2.446      0.014       0.003       0.032
C(position)[T.rb]                 0.0119      0.006      1.832      0.067      -0.001       0.025
C(position)[T.te]                -0.0068      0.005     -1.279      0.201      -0.017       0.004
C(position)[T.wr]                -0.0026      0.007     -0.389      0.698      -0.016       0.011
C(height_dummies)[T.tall]         0.0036      0.006      0.642      0.521      -0.007       0.015
team_match[T.True]                0.0020      0.003      0.789      0.430      -0.003       0.007
demean_DVOA                       0.0051      0.004      1.439      0.150      -0.002       0.012
C(race)[T.W]:demean_DVOA          0.0038      0.004      0.981      0.326      -0.004       0.012
youth                             0.0012      0.002      0.725      0.468      -0.002       0.004
oldness                          -0.0007      0.001     -0.872      0.383      -0.002       0.001
Wins                             -0.0002      0.001     -0.395      0.693      -0.001       0.001
standard_salary                  -0.0007      0.004     -0.157      0.875      -0.009       0.007
std_salary_resid                 -0.0002      0.003     -0.046      0.963      -0.007       0.006
demean_race_diff               -8.99e-07    8.7e-05     -0.010      0.992      -0.000       0.000
C(race)[T.W]:demean_race_diff  7.666e-05      0.000      0.702      0.482      -0.000       0.000
total_population               -1.28e-10   1.82e-10     -0.704      0.481   -4.84e-10    2.28e-10
==============================================================================
Omnibus:                      237.438   Durbin-Watson:                   1.958
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              263.086
Skew:                          -0.031   Prob(JB):                     7.44e-58
Kurtosis:                       3.133   Cond. No.                     3.26e+08
==============================================================================

Warnings:
[1] Standard Errors are robust tocluster correlation (cluster)
[2] The condition number is large, 3.26e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""